<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/wtss/wtss.png?raw=true" align="right" width="64"/>

# <span style="color:#336699">Web Time Series Service (WTSS) - Examples</span>
<hr style="border:2px solid #0077b9;">

<div style="text-align: left;">
    <a href="https://nbviewer.jupyter.org/github/brazil-data-cube/code-gallery/blob/master/jupyter/Python/wtss/wtss-examples.ipynb"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
</div>

<br/>

<div style="text-align: center;font-size: 90%;">
    Rennan F. B. Marujo<sup><a href="https://orcid.org/0000-0002-0082-9498"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Matheus Zaglia<sup><a href="https://orcid.org/0000-0001-6181-2158"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe Menino Carlos<sup><a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
    Last Update: March 12, 2021
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> This Jupyter Notebook gives shows how to use the WTSS service to extract time series from <em>Brazil Data Cube</em>' service and how to perform a basic time series manipulation.
</div>

<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is a supplement to the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Vinhas, L.; Queiroz, G. R.; Ferreira, K. R.; Camara, G. <a href="http://www.seer.ufu.br/index.php/revistabrasileiracartografia/article/view/44004" target="_blank">Web Services for Big Earth Observation Data</a>. Revista Brasileira de Cartografia, v. 69, n. 5, 18 maio 2017.
    </div>
</div>

# Python Client API
<hr style="border:1px solid #0077b9;">

To run the examples that will be presented in this Jupyter Notebook, you will need to have installed the Python library [wtss.py](https://github.com/brazil-data-cube/wtss.py). Additionally, for the time series visualization and interpolation examples, you will need to have installed the libraries [NumPy](https://numpy.org/), [Pandas](https://pandas.pydata.org/), [Matplotlib](https://matplotlib.org/), [Plotly](https://plotly.com/python/), [SciPy](https://www.scipy.org/) and [Folium] (http://python-visualization.github.io/folium/).

The installation of the mentioned libraries can be done with the service [Python Package Index (PyPI)](https://pypi.org/), through the command:

In [ ]:
!pip install wtss[matplotlib] numpy pandas matplotlib plotly scipy folium

> For more information on [WTSS client for Python](https://github.com/brazil-data-cube/wtss.py), see the introductory Jupyter Notebook about [Web Time Series Service (WTSS)](./wtss-introduction.ipynb) Introduction notebook.

# Set the service and Search for time series
<hr style="border:1px solid #0077b9;">

Import the WTSS client library:

In [ ]:
from wtss import WTSS

Define the service to be used:

In [ ]:
service = WTSS('https://brazildatacube.dpi.inpe.br/', access_token='change-me')

In [ ]:
service.coverages

Let's access the CBERS-4/AWFI coverage using the `CB4_64_16D_STK-1` product:

In [ ]:
cbers4_coverage = service['CB4_64_16D_STK-1']
cbers4_coverage

CBERS-4/AWFI spectral bands 15 and 16 correspond to the red and near-infrared (NIR) wavelength regions, respectivelly:

In [ ]:
red_band = 'BAND15'
nir_band = 'BAND16'

Let's retrieve the time series for data product `CB4_64_16D_STK-1`, in the location of `latitude -16.817` and `longitude -52.079` from January 1st, 2017 to December 31st, 2019, using the `ts` method:

In [ ]:
time_series = cbers4_coverage.ts(attributes=(red_band, nir_band),
                                 latitude=-16.817,
                                 longitude=-52.079,
                                 start_date="2017-01-01",
                                 end_date="2019-12-31")

# Plot the Time Series
<hr style="border:1px solid #0077b9;">

**Static Visualization**

Once with the time series, one can visualize it using the `plot` method:

In [ ]:
time_series.plot()

**Interactive Visualization**

Another way to visualize the time series is through interactive graphics, which allow dinamically explore the time series patterns.

To do that, we will use the [Plotly](https://plotly.com/python/) and [Pandas](https://pandas.pydata.org/) libraries, which can be imported as:

In [ ]:
import pandas as pd
import plotly.express as px

Let's create a `pandas.DataFrame` containing the temporal attributes for `BAND15` and `BAND16`:

In [ ]:
cbers_df = pd.DataFrame({ 'BAND15': time_series.BAND15, 'BAND16': time_series.BAND16 }, 
                        index = pd.to_datetime(time_series.timeline))

cbers_df

Now we can use the `pandas.DataFrame` to create an interactive graphic:

In [ ]:
fig = px.line(cbers_df, x=cbers_df.index, y=['BAND15', 'BAND16'], title='CBERS-4/AWFI (BAND 15 and 16)', labels={
    'index': 'Date',
    'value': 'Spectral Reflectance (scaled)'
})

fig.update_xaxes(rangeslider_visible=True)
fig.show()

# Removing clouds from Time Series
<hr style="border:1px solid #0077b9;">

When we work with satellite image time series there are external influences that can bring noise to the values of the series, distorting them. A common example of noise is the observation of clouds, which change the spectral behavior of targets, making them often unidentifiable. In Figure 1, images from the Sentinel-2/MSI sensor satellite of the `20LKP` tile are shown, where it is possible to observe that, for the same region, in close dates, the presence of clouds.

<div align="center">
    <figcaption><strong>Figura 1</strong> - Imagens Sentinel-2/MSI (Tile 20LKP) com influência de Nuvem</figcaption>
    <img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/wtss/sentinel-2-clouds.png?raw=true" align="center" width="620rem"/>
    <br>
    <strong>Fonte</strong>: Simoes <i>et al.</i> (2021)
</div>

In the context of satellite image time series, a possible approach for the treatment of this type of noise is to replace the time series points that have cloud influence by interpolated values.

For this, it is necessary to have data available to identify the influence of the cloud at each point in the time series. When considering the BDC data products, this approach is possible since the data cubes are produced with masks that identify, *pixel* to *pixel*, the cloud influence on the images.

Therefore, in this second example, we will use the cloud mask provided in the BDC data products, together with the linear interpolation technique, to remove the cloud influence from the time series.

To start, first lets look at the `CB4_64_16D_STK-1` *Coverage* metadata:

In [ ]:
cbers4_coverage

In this metadata, we can see the attribute `CMASK`, which stands for cloud mask and uses the following values:

- `0`: *Pixel* no data;
- `127`: *Pixel* clear observation (no cloud);
- `255`: *Pixel* cloud observation.

Let's retrieve the time series, for the location `latitude -12.0`, `longitude -53.989` from `January 1, 2017` to `December 31, 2019`.

In [ ]:
cb4_timeseries_cmask = cbers4_coverage.ts(

    attributes = ('CMASK'),

    latitude = -12.0,
    longitude = -53.989,

    start_date = "2017-01-01",
    end_date = "2019-12-31"
)

cb4_timeseries_cmask

Let's preview the returned values. To make the visualization simpler, let's remove the values that are duplicated:

In [ ]:
set(cb4_timeseries_cmask.values('CMASK'))

Note that, among the values returned, there are *pixels* that have values `127` and `255`. This indicates that certain values have cloud (`255`) and may be removed.

To present the example of interpolation, let's make use of the `NDVI` attribute. So, first let's do a new time series extraction, considering now both attributes, `NDVI` and `CMASK`:

In [ ]:
cb4_timeseries = cbers4_coverage.ts(

    attributes = ('NDVI', 'CMASK'),

    latitude = -12.0,
    longitude = -53.989,

    start_date = "2017-01-01",
    end_date = "2019-12-31"

)

Let's perform this operation step by step. So, first let's retrieve the timeline from the extracted time series and then transform them into `datetime` type objects. This can be done with the command below:

<div style="text-align: center; margin-left: 25%; margin-right: 25%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Tip:</b> The timeline to <em>datetime</em> conversion operation is necessary since, although the <em>cb4_timeseries.timeline</em> attribute stores a list of dates, in Python, the values in this list are just text and are not interpreted by the language as valid dates. This can make it difficult to manipulate this data, especially when considering aggregations and temporal selections of the data using the <em>Pandas</em> library.
</div>

In [ ]:
ndvi_timeline = pd.to_datetime(cb4_timeseries.timeline)
ndvi_timeline

Now, let's store the values of the `NDVI` and `CMASK` attributes of the extracted time series in a `numpy.array`. This will be done to facilitate the manipulation of this data.

In [ ]:
import numpy as np

ndvi_data = np.array(cb4_timeseries.NDVI)
ndvi_data

cmask_data = np.array(cb4_timeseries.CMASK)
cmask_data

Now, we need to adopt some strategy to make the values that have cloud influence be marked as `nan` in the dataset.

As the values of both attributes are stored in a `numpy.array`, we can then use the features of [numpy](https://numpy.org/), to manipulate this data.

Considering this, we are going to adopt a strategy in which we will apply a transformation to the `CMASK` data, so that every position in the array that represents cloud influence will be converted to `nan`, while the other values will be converted to `1`. With this approach, we will be able to multiply the array resulting from the transformation, with the data of other attributes, which will make the positions with cloud influence become `nan` as well.

To be clear, let's perform this operation. First, with the code below, let's convert the values of `CMASK`

In [ ]:
cmask_data = np.where(cmask_data == 255, np.nan, 1)
cmask_data

Note that now, there are some `nan` values in the `cmask` array, while all others are equal to `1`. By multiplying this mask array, with the `NDVI` array, we have the following effect:

In [ ]:
ndvi_data * cmask_data

Note that among the values of `NDVI` in the time series, some values were transformed into `nan`. Now, we can perform the interpolation of our time series.

For that, we're going to put this data inside a `pandas.DataFrame`. As we only have one data attribute, `NDVI`, let's multiply it with the mask array and store it in a column of the `DataFrame`, named `data`. The index (Name of the lines of the `DataFrame`), will be each of the dates of the timeline (Converted to datetime in the previous step).

The command below creates the `DataFrame` with the specified characteristics:

In [ ]:
ndvi_masked_data = pd.DataFrame({ 'data': ndvi_data * cmask_data }, index = pd.to_datetime(ndvi_timeline))
ndvi_masked_data

If we look in the `DataFrame` for the lines that have `na` values, we should find some of them:

In [ ]:
ndvi_masked_data[ndvi_masked_data['data'].isna()]

Now, using the `interpolate` method, available in `pandas.DataFrame`, let's interpolate our data:

In [ ]:
ndvi_masked_data_interpolated = ndvi_masked_data.interpolate()
ndvi_masked_data_interpolated

The interpolation result was saved in the `ndvi_masked_data_interpolated` variable. When querying this variable for the existence of `nan` values, no value should be returned:

In [ ]:
ndvi_masked_data_interpolated[ndvi_masked_data_interpolated['data'].isna()]

Done! We interpolate our data. To visualize the result of this operation and the difference caused by this change in the series, below, let's visualize the original series and the interpolated series.

As the [Matplotlib](https://matplotlib.org/) library will be used in this view, it must be imported. This can be done with the code below:

In [ ]:
import matplotlib.pyplot as plt

We can then compare time series

In [ ]:
plt.figure(dpi = 120)

plt.plot(ndvi_data, color='gray', linestyle='dashed', label = 'Original')
plt.plot(ndvi_masked_data_interpolated['data'].values, color='blue', label = 'Interpolated')

plt.title('Comparison of Time Series with and without interpolation')
plt.legend()
plt.grid(True)
plt.show()

## Time series smoothing
<hr style="border:1px solid #0077b9;">

In addition to cloud noise, there are many other factors that can influence the time series values ​​of Remote Sensing images. For these cases, a solution can be to apply filters to smooth the time series.

There are several smoothing algorithms that can be used, in this example the smoothing of the time series will be presented using the `Savitzky Golay` filter. The time series used in this example will be the same interpolated in the previous step.

For the example, the `Savitzky Golay` filter implemented in the library [SciPy](https://www.scipy.org/) will be used. Importing the filter function can be done as follows:

In [ ]:
from scipy.signal import savgol_filter

With the filter function imported, let's apply it to the interpolated data:

In [ ]:
median_smoothed = savgol_filter(ndvi_masked_data_interpolated['data'], window_length = 9, polyorder = 2)

<div style="text-align: center; margin-left: 25%; margin-right: 25%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Note:</b> The <em>Savitz Golay</em> algorithm uses <b>window_length</b> and the <b>polynomial order</b> used as parameters. You can change these values ​​to see the impacts on the smoothing of the series. Additionally, if you are more interested in time series smoothing, it is recommended to read <a href="https://e-sensing.github.io/sitsbook/working-with-time-series.html#filtering -techniques-for-time-series" target="_blank"> "Section 3.5 - Filtering techniques for time series" of the <em>sits</em></a> user manual
</div>

To compare the results, in the same way as for the interpolation, let's create a plot that allows us to visualize the changes:

In [ ]:
plt.figure(dpi = 120)

plt.plot(ndvi_masked_data_interpolated['data'].values, color='gray', linestyle='dashed', label = 'Interpolated')
plt.plot(median_smoothed, color='blue', label = 'Interpolated and smoothed')

plt.title('Comparison of Time Series with and without interpolation')
plt.legend()
plt.grid(True)
plt.show()

# Calculate Median Time Series
<hr style="border:1px solid #0077b9;">

So far, all examples have made use of time series varying only in the number of attributes. Thus, in this example, a set of time series will be used in order to extract a median time series.

The first thing to do then, is to determine the points that will be used. In this case, for the extraction of time series, we will generate points in an agricultural region in the state of Mato Grosso. These points will be generated considering the following criteria:

- Longitude: Fixed at the value `-53,989`;
- Latitude: Variable from `-16.9000` to `-16.9075` at a rate of `-0.0015`.

This will generate five points. These will be used to extract time series from the previously selected *Coverage*, considering data from `January 1, 2017` to `December 31, 2019`. Only the NDVI vegetation index attribute will be used.

Let's start with generating the points, which can be done with the code below:

In [ ]:
longitude = -53.989  # Fixado
locations = []  # variável

for latitude in np.arange(-16.9000,-16.9075,-0.0015):
    locations.append(( latitude, longitude ))

At the end of the operation, we have a set of locations stored in the `locations` variable:

In [ ]:
locations

To more clearly present the region from which the time series will be collected, let's place the generated points on an interactive map:

In [ ]:
import folium

Creating the map:

In [ ]:
m = folium.Map(
    location = [-16.9, -53.989],
    zoom_start = 15,
)

tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)

Finally, let's add the points to the map and then do the visualization:

In [ ]:
for location in locations:
    folium.Circle(
        fill=True,  # Preenchimento
        color = 'orange',  # Cor
        location=location,  # Localização
    ).add_to(m)

m

Now, let's extract the time series from each of these points.

In [ ]:
agriculture_time_series = []

for location in locations:
    latitude, longitude = location

    time_series = cbers4_coverage.ts(attributes=('NDVI'),
                                     latitude=float(latitude), longitude=float(longitude),
                                     start_date="2017-01-01", end_date="2019-12-31")

    agriculture_time_series.append(time_series.values('NDVI'))

As we had 6 points, we must have saved 6 time series. Let's check it out by looking at the number of elements in the `agriculture_time_series` list. This can be done with the `len` function, as shown in the command below:

In [ ]:
len(agriculture_time_series)

Now, using the `median` function from the [NumPy](https://numpy.org/) library, let's calculate the median value for each point in the time series:

<div style="text-align: center; margin-left: 25%; margin-right: 25%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Tip:</b> The operation <em>median</em> will use the parameter <em>axis=0</em> so that the median is calculated considering a set of points in each instant of time, so that the median of each instant of time is the median value of 5 values ​​(One of each point).
</div>

In [ ]:
median = np.median(agriculture_time_series, axis = 0)
median

Now, let's visualize each of the original time series and compare them to the median:

In [ ]:
plt.figure(dpi = 120)

#
# Criando plot das séries temporais originais
#
for i in range(len(agriculture_time_series)):
    plt.plot(agriculture_time_series[i], color = 'grey', alpha = .2)

plt.plot(median, color='blue', linewidth = 1)

plt.grid(True)
plt.show()

# References
<hr style="border:1px solid #0077b9;">

- [Python Client Library for Web Time Series Service - User Guide](https://wtss.readthedocs.io/en/latest/index.html)


- [Python Client Library for Web Time Series Service - GitHub Repository](https://github.com/brazil-data-cube/wtss.py)


- [WTSS OpenAPI 3 Specification](https://github.com/brazil-data-cube/wtss-spec)


- VINHAS, L.; QUEIROZ, G. R.; FERREIRA, K. R.; CÂMARA, G. [Web Services for Big Earth Observation Data](http://www.seer.ufu.br/index.php/revistabrasileiracartografia/article/view/44004). Revista Brasileira de Cartografia, v. 69, n. 5, 18 maio 2017.

- Simoes, R., Camara, G., Souza, F., Santos, L., Andrade, P. R., Peletier, C., Carvalho, A., Ferreira, K., Queiroz, G., & Maus, V. (2021). sits: Data Analysis and Machine Learning on Earth Observation Data Cubes with Satellite Image Time Series. Online.

# See also the following Jupyter Notebooks
<hr style="border:1px solid #0077b9;">

* [Introduction to the Web Time Series Service (WTSS)](./wtss-introduction.ipynb)